<a href="https://colab.research.google.com/github/atik-bd/MSc-BA/blob/main/ML_Used_Car_Price_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Preparation

In [ ]:
# Mounting the packages into Colab
from pandas import read_csv, DataFrame, get_dummies,to_datetime
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn import linear_model
from  plotly import figure_factory

In [ ]:
# Mounting the dataset into Colab
data1=read_csv('/content/processes2.csv')
data1.info() #Found no mssing data in any features

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2095 entries, 0 to 2094
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          2095 non-null   int64  
 1   name                2095 non-null   object 
 2   year                2095 non-null   int64  
 3   selling_price       2095 non-null   int64  
 4   km_driven           2095 non-null   int64  
 5   fuel                2095 non-null   object 
 6   seller_type         2095 non-null   object 
 7   transmission        2095 non-null   object 
 8   owner               2095 non-null   object 
 9   seats               2095 non-null   int64  
 10  max_power (in bph)  2095 non-null   float64
 11  Mileage Unit        2095 non-null   object 
 12  Mileage             2095 non-null   float64
 13  Engine (CC)         2095 non-null   int64  
dtypes: float64(2), int64(6), object(6)
memory usage: 229.3+ KB


In [ ]:
# Checking the first ten rows of the dataset
data1.head(10)

,Unnamed: 0,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,seats,max_power (in bph),Mileage Unit,Mileage,Engine (CC)
0,0,Maruti,2014,450000,145500,Diesel,Individual,Manual,First Owner,5,74.00,kmpl,23.40,1248
1,2,Hyundai,2010,225000,127000,Diesel,Individual,Manual,First Owner,5,90.00,kmpl,23.00,1396
2,4,Hyundai,2017,440000,45000,Petrol,Individual,Manual,First Owner,5,81.86,kmpl,20.14,1197
3,7,Toyota,2011,350000,90000,Diesel,Individual,Manual,First Owner,5,67.10,kmpl,23.59,1364
4,8,Ford,2013,200000,169000,Diesel,Individual,Manual,First Owner,5,68.10,kmpl,20.00,1399
5,15,Hyundai,2012,500000,53000,Diesel,Individual,Manual,Second Owner,5,90.00,kmpl,23.00,1396
6,16,Maruti,2002,150000,80000,Petrol,Individual,Manual,Second Owner,5,46.30,kmpl,19.70,796
7,17,Hyundai,2016,680000,100000,Diesel,Individual,Manual,First Owner,5,88.73,kmpl,22.54,1396
8,18,Mahindra,2011,174000,100000,Diesel,Individual,Manual,Second Owner,5,64.10,kmpl,21.00,1461
9,21,Maruti,2012,600000,72000,Diesel,Individual,Manual,First Owner,5,88.80,kmpl,21.50,1248


In [ ]:
# Checking the numbers of rows and columns (features) in the dataset
data1.shape

(2095, 14)

In [ ]:
# Checking the basic statistics for the dataset
data1.describe()

,Unnamed: 0,year,selling_price,km_driven,seats,max_power (in bph),Mileage,Engine (CC)
count,2095.000000,2095.000000,2.095000e+03,2095.000000,2095.000000,2095.000000,2095.000000,2095.000000
mean,3155.537947,2014.052506,4.272799e+05,67413.593795,5.096897,77.410427,21.397928,1205.078282
std,1791.039215,3.273451,2.326698e+05,41580.846333,0.457000,19.884938,1.209234,245.058696
min,0.000000,2000.000000,4.595700e+04,1000.000000,4.000000,35.000000,19.600000,624.000000
25%,1589.500000,2012.000000,2.500000e+05,35000.000000,5.000000,67.040000,20.360000,998.000000
50%,3185.000000,2014.000000,3.900000e+05,60000.000000,5.000000,74.000000,21.100000,1248.000000
75%,4709.500000,2017.000000,5.555000e+05,90000.000000,5.000000,88.730000,22.540000,1396.000000
max,6256.000000,2020.000000,1.594000e+06,270000.000000,8.000000,126.320000,23.590000,1598.000000


# Feature Selection (overfitting avoidance)

In [ ]:
# dropping manually irrelevant features
data1 = data1.drop(['Unnamed: 0','Mileage Unit'], axis = 1)
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2095 entries, 0 to 2094
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   name                2095 non-null   object 
 1   year                2095 non-null   int64  
 2   selling_price       2095 non-null   int64  
 3   km_driven           2095 non-null   int64  
 4   fuel                2095 non-null   object 
 5   seller_type         2095 non-null   object 
 6   transmission        2095 non-null   object 
 7   owner               2095 non-null   object 
 8   seats               2095 non-null   int64  
 9   max_power (in bph)  2095 non-null   float64
 10  Mileage             2095 non-null   float64
 11  Engine (CC)         2095 non-null   int64  
dtypes: float64(2), int64(5), object(5)
memory usage: 196.5+ KB


# Data Encoding

In [ ]:
# Checking features with object data type
print("transmission:", data1['transmission'].unique())
print("\nowner:", data1['owner'].unique())
print("\nname:", data1['name'].unique())
print("\nfuel:", data1['fuel'].unique())
print("\nseller_type:", data1['seller_type'].unique())

transmission: ['Manual' 'Automatic']

owner: ['First Owner' 'Second Owner' 'Third Owner' 'Fourth & Above Owner'
 'Test Drive Car']

name: ['Maruti' 'Hyundai' 'Toyota' 'Ford' 'Mahindra' 'Tata' 'Volkswagen'
 'Renault' 'Chevrolet' 'Honda']

fuel: ['Diesel' 'Petrol' 'CNG' 'LPG']

seller_type: ['Individual' 'Dealer' 'Trustmark Dealer']


In [ ]:
# Binary Encoding_ converting categorical features into numerical binary values
data1['transmission']=data1['transmission'].map({"Automatic":1,"Manual":0})

In [ ]:
# Label encoding for ordinal categorical variables
data1['owner']=data1['owner'].map({'First Owner':4,'Second Owner':3,'Third Owner':2,'Fourth & Above Owner':1,'Test Drive Car':0})

In [ ]:
# One-Hot Encoding_ converts categorical features into multiple binary columns, drop first column to reduce feature
data1=get_dummies(data1,['name','fuel','seller_type'],dtype=int,drop_first=True)
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2095 entries, 0 to 2094
Data columns (total 23 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   year                          2095 non-null   int64  
 1   selling_price                 2095 non-null   int64  
 2   km_driven                     2095 non-null   int64  
 3   transmission                  2095 non-null   int64  
 4   owner                         2095 non-null   int64  
 5   seats                         2095 non-null   int64  
 6   max_power (in bph)            2095 non-null   float64
 7   Mileage                       2095 non-null   float64
 8   Engine (CC)                   2095 non-null   int64  
 9   name_Ford                     2095 non-null   int64  
 10  name_Honda                    2095 non-null   int64  
 11  name_Hyundai                  2095 non-null   int64  
 12  name_Mahindra                 2095 non-null   int64  
 13  nam

# Feature Reduction using Heatmap

In [ ]:
#Heatmap
correlation=data1.corr()
f=figure_factory.create_annotated_heatmap(correlation.values,list(correlation.columns),list(correlation.columns),correlation.round(2).values,showscale=True)
f.show()

In [ ]:
# Droping highly correlated features
data1 = data1.drop(['max_power (in bph)'], axis = 1)
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2095 entries, 0 to 2094
Data columns (total 22 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   year                          2095 non-null   int64  
 1   selling_price                 2095 non-null   int64  
 2   km_driven                     2095 non-null   int64  
 3   transmission                  2095 non-null   int64  
 4   owner                         2095 non-null   int64  
 5   seats                         2095 non-null   int64  
 6   Mileage                       2095 non-null   float64
 7   Engine (CC)                   2095 non-null   int64  
 8   name_Ford                     2095 non-null   int64  
 9   name_Honda                    2095 non-null   int64  
 10  name_Hyundai                  2095 non-null   int64  
 11  name_Mahindra                 2095 non-null   int64  
 12  name_Maruti                   2095 non-null   int64  
 13  nam

# Data Splitting and Scaling

In [ ]:
# Dividing dataset into label and feature sets
X = data1.drop(['selling_price'], axis = 1) # Features
Y = data1['selling_price'] # Label
print(X.shape)
print(Y.shape)

(2095, 21)
(2095,)


In [ ]:
# Data Scaling
x_scaled = StandardScaler().fit_transform(X)
x_scaled_n= DataFrame(x_scaled,columns=X.columns)
x_scaled_n.head()

,year,km_driven,transmission,owner,seats,Mileage,Engine (CC),name_Ford,name_Honda,name_Hyundai,...,name_Maruti,name_Renault,name_Tata,name_Toyota,name_Volkswagen,fuel_Diesel,fuel_LPG,fuel_Petrol,seller_type_Individual,seller_type_Trustmark Dealer
0,-0.016044,1.878390,-0.222709,0.670051,-0.21208,1.656049,0.175191,-0.280686,-0.057901,-0.612104,...,1.129027,-0.235405,-0.257212,-0.157959,-0.236532,0.974083,-0.021853,-0.955637,0.302844,-0.057901
1,-1.238287,1.433367,-0.222709,0.670051,-0.21208,1.325182,0.779272,-0.280686,-0.057901,1.633708,...,-0.885719,-0.235405,-0.257212,-0.157959,-0.236532,0.974083,-0.021853,-0.955637,0.302844,-0.057901
2,0.900639,-0.539165,-0.222709,0.670051,-0.21208,-1.040518,-0.032973,-0.280686,-0.057901,1.633708,...,-0.885719,-0.235405,-0.257212,-0.157959,-0.236532,-1.026607,-0.021853,1.046422,0.302844,-0.057901
3,-0.932726,0.543322,-0.222709,0.670051,-0.21208,1.813211,0.648660,-0.280686,-0.057901,-0.612104,...,-0.885719,-0.235405,-0.257212,6.330753,-0.236532,0.974083,-0.021853,-0.955637,0.302844,-0.057901
4,-0.321605,2.443689,-0.222709,0.670051,-0.21208,-1.156321,0.791517,3.562697,-0.057901,-0.612104,...,-0.885719,-0.235405,-0.257212,-0.157959,-0.236532,0.974083,-0.021853,-0.955637,0.302844,-0.057901


# Feature Reduction using P-Stats

In [ ]:
import statsmodels.api as sm
p_model=sm.add_constant(x_scaled_n)
p_model2=sm.OLS(Y,p_model).fit()
print(p_model2.summary())
print("Best fit R-squared:", p_model2.rsquared)
print("Best fit Adjusted R-squared:", p_model2.rsquared_adj)

                            OLS Regression Results                            
Dep. Variable:          selling_price   R-squared:                       0.737
Model:                            OLS   Adj. R-squared:                  0.735
Method:                 Least Squares   F-statistic:                     277.0
Date:                Fri, 25 Apr 2025   Prob (F-statistic):               0.00
Time:                        21:37:41   Log-Likelihood:                -27461.
No. Observations:                2095   AIC:                         5.497e+04
Df Residuals:                    2073   BIC:                         5.509e+04
Df Model:                          21                                         
Covariance Type:            nonrobust                                         
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
const           

In [ ]:
# Finding significant features based on p value
p_values= p_model2.pvalues
significant_features=p_values[p_values<=0.05]
print(significant_features)

const                      0.000000e+00
year                      3.301899e-218
km_driven                  1.351227e-21
transmission               1.644582e-04
owner                      9.654449e-04
seats                      9.922635e-06
Mileage                    1.543823e-23
Engine (CC)                3.345183e-70
name_Ford                  1.594293e-02
name_Honda                 5.862961e-08
name_Hyundai               2.016775e-07
name_Maruti                3.922892e-09
name_Renault               1.865310e-03
name_Tata                  4.071639e-02
name_Toyota                1.456679e-07
fuel_Diesel                2.261047e-18
fuel_Petrol                3.174804e-05
seller_type_Individual     1.288448e-04
dtype: float64


In [ ]:
# Adjusting the dataset with significant features and dropping added constant
x_scaled_n = p_model[significant_features.index]
x_scaled_n = x_scaled_n.drop(['const'], axis = 1)
x_scaled_n.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2095 entries, 0 to 2094
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   year                    2095 non-null   float64
 1   km_driven               2095 non-null   float64
 2   transmission            2095 non-null   float64
 3   owner                   2095 non-null   float64
 4   seats                   2095 non-null   float64
 5   Mileage                 2095 non-null   float64
 6   Engine (CC)             2095 non-null   float64
 7   name_Ford               2095 non-null   float64
 8   name_Honda              2095 non-null   float64
 9   name_Hyundai            2095 non-null   float64
 10  name_Maruti             2095 non-null   float64
 11  name_Renault            2095 non-null   float64
 12  name_Tata               2095 non-null   float64
 13  name_Toyota             2095 non-null   float64
 14  fuel_Diesel             2095 non-null   

# Linear Regression

In [ ]:
r, c=x_scaled_n.shape
print("r, c:", x_scaled_n.shape)

r, c: (2095, 17)


In [ ]:
# Linear Regression (LR) with no penalty
LinearRegression = linear_model.SGDRegressor(random_state = 1, penalty = None) # model building
Hparameter = {'eta0': [.001, .01, .1, 1], 'max_iter':[5000,10000,20000,30000]}
grid_search = GridSearchCV(estimator=LinearRegression, param_grid=Hparameter, scoring='r2', cv=5)
grid_search.fit(x_scaled_n,Y)

best_parameters_LR = grid_search.best_params_
print("Best parameters LR: ", best_parameters_LR)
best_model_LR = grid_search.best_estimator_
print("Intercept β0: ", best_model_LR.intercept_)
best_result_LR = grid_search.best_score_
print("Best result LR: ", best_result_LR)
modified_r2_LR = 1-(1-best_result_LR)*(4/5*r-1)/(4/5*r-c-1)# 4/5*r is number of rows in training set, c is number of columns
print("modified_r2_LR: ", modified_r2_LR)
print(DataFrame(zip(x_scaled_n.columns, best_model_LR.coef_), columns=['Features','Coefficients']).sort_values(by=['Coefficients'],ascending=False))

Best parameters LR:  {'eta0': 0.001, 'max_iter': 5000}
Intercept β0:  [427279.01643989]
Best result LR:  0.7284018149625375
modified_r2_LR:  0.7256170326069061
                  Features   Coefficients
0                     year  130662.366152
6              Engine (CC)   95286.493951
14             fuel_Diesel   94788.312625
10             name_Maruti   85334.851731
9             name_Hyundai   66149.398209
15             fuel_Petrol   27971.273068
13             name_Toyota   25990.191304
11            name_Renault   16997.367187
8               name_Honda   16222.978155
4                    seats   13018.880652
7                name_Ford   12389.608950
2             transmission   10869.739985
3                    owner    9488.135346
12               name_Tata    8126.795817
16  seller_type_Individual   -9986.952727
1                km_driven  -33134.873474
5                  Mileage  -34143.816721


In [ ]:
# Linear Regression (LR) with L1
LinearRegression1 = linear_model.SGDRegressor(random_state = 1, penalty = 'l1') # model building
Hparameter1 = {'eta0': [.001, .01, .1, 1], 'max_iter':[5000,10000,20000,30000],'alpha': [.01, .1, 1,10, 100]}
grid_search1 = GridSearchCV(estimator=LinearRegression1, param_grid=Hparameter1, scoring='r2', cv=5)
grid_search1.fit(x_scaled_n,Y)

best_parameters_LR1 = grid_search1.best_params_
print("Best parameters LR1: ", best_parameters_LR1)
best_model_LR1 = grid_search1.best_estimator_
print("Intercept β0: ", best_model_LR1.intercept_)
best_result_LR1 = grid_search1.best_score_
print("Best result LR1: ", best_result_LR1)
modified_r2_LR1 = 1-(1-best_result_LR1)*(4/5*r-1)/(4/5*r-c-1)# 4/5*r is number of rows in training set, c is number of columns
print("modified_r2_LR1: ", modified_r2_LR1)
print(DataFrame(zip(x_scaled_n.columns, best_model_LR1.coef_), columns=['Features','Coefficients']).sort_values(by=['Coefficients'],ascending=False))

Best parameters LR1:  {'alpha': 0.01, 'eta0': 0.001, 'max_iter': 5000}
Intercept β0:  [427279.01645211]
Best result LR1:  0.7284017971486628
modified_r2_LR1:  0.7256170146103801
                  Features   Coefficients
0                     year  130662.354516
6              Engine (CC)   95286.460069
14             fuel_Diesel   94787.879036
10             name_Maruti   85334.666436
9             name_Hyundai   66149.253479
15             fuel_Petrol   27970.862043
13             name_Toyota   25990.131389
11            name_Renault   16997.292735
8               name_Honda   16222.952982
4                    seats   13018.882609
7                name_Ford   12389.531550
2             transmission   10869.734478
3                    owner    9488.131051
12               name_Tata    8126.713761
16  seller_type_Individual   -9986.949589
1                km_driven  -33134.866163
5                  Mileage  -34143.783748


In [ ]:
# Linear Regression (LR) with L2
LinearRegression2 = linear_model.SGDRegressor(random_state = 1, penalty = 'l2') # model building
Hparameter2 = {'eta0': [.001, .01, .1, 1], 'max_iter':[5000,10000,20000,30000],'alpha': [.01, .1, 1,10, 100]}
grid_search2 = GridSearchCV(estimator=LinearRegression2, param_grid=Hparameter2, scoring='r2', cv=5)
grid_search2.fit(x_scaled_n,Y)

best_parameters_LR2 = grid_search2.best_params_
print("Best parameters LR2: ", best_parameters_LR2)
best_model_LR2 = grid_search2.best_estimator_
print("Intercept β0: ", best_model_LR2.intercept_)
best_result_LR2 = grid_search2.best_score_
print("Best result LR2: ", best_result_LR2)
modified_r2_LR2 = 1-(1-best_result_LR2)*(4/5*r-1)/(4/5*r-c-1)# 4/5*r is number of rows in training set, c is number of columns
print("modified_r2_LR2: ", modified_r2_LR2)
print(DataFrame(zip(x_scaled_n.columns, best_model_LR2.coef_), columns=['Features','Coefficients']).sort_values(by=['Coefficients'],ascending=False))

Best parameters LR2:  {'alpha': 0.01, 'eta0': 0.01, 'max_iter': 5000}
Intercept β0:  [427614.01429588]
Best result LR2:  0.7269548392177173
modified_r2_LR2:  0.7241552205607216
                  Features   Coefficients
0                     year  127604.760684
6              Engine (CC)   93239.761987
10             name_Maruti   75022.232807
14             fuel_Diesel   72728.266007
9             name_Hyundai   60153.898544
13             name_Toyota   22149.910276
8               name_Honda   15139.572702
11            name_Renault   14410.821333
4                    seats   13196.989606
2             transmission   12012.334920
3                    owner   10260.583159
15             fuel_Petrol    8603.665490
7                name_Ford    8575.338704
12               name_Tata    5350.483193
16  seller_type_Individual  -10580.743538
5                  Mileage  -32377.681419
1                km_driven  -32745.393408


In [ ]:
# Linear Regression (LR) with Elasticnet
LinearRegression3 = linear_model.SGDRegressor(random_state = 1, penalty = 'elasticnet') # model building
Hparameter3 = {'eta0': [.001, .01, .1, 1], 'max_iter':[5000,10000,20000,30000],'alpha': [.01, .1, 1,10, 100],'l1_ratio': [0,0.25,0.5,0.75,1]}
grid_search3 = GridSearchCV(estimator=LinearRegression3, param_grid=Hparameter3, scoring='r2', cv=5)
grid_search3.fit(x_scaled_n,Y)

best_parameters_LR3 = grid_search3.best_params_
print("Best parameters LR3: ", best_parameters_LR3)
best_model_LR3 = grid_search3.best_estimator_
print("Intercept β0: ", best_model_LR3.intercept_)
best_result_LR3 = grid_search3.best_score_
print("Best result LR3: ", best_result_LR3)
modified_r2_LR3 = 1-(1-best_result_LR3)*(4/5*r-1)/(4/5*r-c-1)# 4/5*r is number of rows in training set, c is number of columns
print("modified_r2_LR3: ", modified_r2_LR3)
print(DataFrame(zip(x_scaled_n.columns, best_model_LR3.coef_), columns=['Features','Coefficients']).sort_values(by=['Coefficients'],ascending=False))

Best parameters LR3:  {'alpha': 0.01, 'eta0': 0.001, 'l1_ratio': 1, 'max_iter': 5000}
Intercept β0:  [427279.01645211]
Best result LR3:  0.7284017971486628
modified_r2_LR3:  0.7256170146103801
                  Features   Coefficients
0                     year  130662.354516
6              Engine (CC)   95286.460069
14             fuel_Diesel   94787.879036
10             name_Maruti   85334.666436
9             name_Hyundai   66149.253479
15             fuel_Petrol   27970.862043
13             name_Toyota   25990.131389
11            name_Renault   16997.292735
8               name_Honda   16222.952982
4                    seats   13018.882609
7                name_Ford   12389.531550
2             transmission   10869.734478
3                    owner    9488.131051
12               name_Tata    8126.713761
16  seller_type_Individual   -9986.949589
1                km_driven  -33134.866163
5                  Mileage  -34143.783748


# Support Vector Regression

In [ ]:
from sklearn.svm import SVR
SVRegressor = SVR()
cv=10
Hparameters4 = {'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'C': [100,1000,10000], 'epsilon': [100,1000,10000]}
grid_search4 = GridSearchCV(estimator=SVRegressor, param_grid=Hparameters4, scoring='r2')
grid_search4.fit(x_scaled_n, Y)

best_parameters_SVR = grid_search4.best_params_
print("Best parameters SVR: ", best_parameters_SVR)
best_result_SVR = grid_search4.best_score_
print("Best result SVR: ", best_result_SVR)

modified_r2_SVR = 1-(1-best_result_SVR)*((cv-1)/cv*r-1)/((cv-1)/cv*r-c-1)
# 9/10*r is number of rows in training set, c is number of columns
print("modified_r2 SVR: ", modified_r2_SVR)

Best parameters SVR:  {'C': 10000, 'epsilon': 10000, 'kernel': 'linear'}
Best result SVR:  0.7033318425411857
modified_r2 SVR:  0.7006312488722166


In [ ]:
# Assuming 'linear' kernel is best
if best_parameters_SVR['kernel'] == 'linear':
    print(DataFrame(zip(x_scaled_n.columns, grid_search4.best_estimator_.dual_coef_[0]), columns=['Features','Coefficients']).sort_values(by=['Coefficients'],ascending=False))
else:
    print("Coefficients are not directly accessible for non-linear kernels in SVR.")

                  Features  Coefficients
0                     year       10000.0
6              Engine (CC)       10000.0
7                name_Ford       10000.0
5                  Mileage       10000.0
13             name_Toyota       10000.0
12               name_Tata       10000.0
11            name_Renault       10000.0
10             name_Maruti       10000.0
9             name_Hyundai       10000.0
16  seller_type_Individual       10000.0
8               name_Honda      -10000.0
3                    owner      -10000.0
4                    seats      -10000.0
1                km_driven      -10000.0
2             transmission      -10000.0
14             fuel_Diesel      -10000.0
15             fuel_Petrol      -10000.0


# Random Forest Regression

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from pandas import Series
RF_Regressor1 = RandomForestRegressor(criterion='squared_error', max_features='sqrt', random_state=1)
no_Trees = {'n_estimators': [200,300,1000,2000]}
grid_search5 = GridSearchCV(estimator=RF_Regressor1, param_grid=no_Trees, scoring='r2', cv=5)
grid_search5.fit(x_scaled_n, Y)

best_parameters_RFR = grid_search5.best_params_
print("Best parameters RFR: ", best_parameters_RFR)

best_result_RFR = grid_search5.best_score_
print("Best result RFR: ", best_result_RFR)
modified_r2_RFR = 1-(1-best_result_RFR)*(4/5*r-1)/(4/5*r-c-1)
print("modified_r2 RFR: ", modified_r2_RFR)
Important_feature = Series(grid_search5.best_estimator_.feature_importances_, index=list(x_scaled_n)).sort_values(ascending=False) # Getting feature importances list for the best model
print(Important_feature)

Best parameters RFR:  {'n_estimators': 300}
Best result RFR:  0.8840685231209211
modified_r2 RFR:  0.8828798409092539
year                      0.343625
Engine (CC)               0.201412
Mileage                   0.120388
km_driven                 0.109229
fuel_Diesel               0.059036
fuel_Petrol               0.042587
owner                     0.034325
name_Hyundai              0.016898
seats                     0.014794
name_Maruti               0.012771
name_Tata                 0.010022
transmission              0.008707
name_Ford                 0.008496
seller_type_Individual    0.006810
name_Renault              0.005168
name_Toyota               0.003361
name_Honda                0.002371
dtype: float64


# # Prediction Using New Data.

In [ ]:
# We will use the Linear Regression Model with Elasticnet to predict a new data
import joblib
joblib.dump(best_model_LR3, "model.pkl")

['model.pkl']

In [ ]:
#columns = ['year', 'km_driven', 'transmission', 'owner', 'seats', 'Mileage', 'Engine (CC)', 'name_Ford', 'name_Honda', 'name_Hyundai',
#           'name_Maruti', 'name_Renault', 'name_Tata', 'name_Toyota', 'fuel_Diesel', 'fuel_Petrol', 'seller_type_Individual']
My_model = joblib.load("model.pkl")
x=[[1,1,1,1,1,1,1,1,1,0,0,1,0,1,0,0,1]]
predict=My_model.predict(x)[0]
predict=round(My_model.predict(x)[0],2)
print(predict)

680938.89


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning:

X does not have valid feature names, but SGDRegressor was fitted with feature names

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning:

X does not have valid feature names, but SGDRegressor was fitted with feature names



In [ ]:
# testing the model with random rows of original data
data1['selling_price'][100:110]

,selling_price
100,140000
101,225000
102,650000
103,600000
104,150000
105,535000
106,300000
107,240000
108,409999
109,150000


In [ ]:
selling_price=My_model.predict(x_scaled_n[100:110]) # Predict prices for 10 samples
DataFrame(selling_price) #Display as DataFrame

,0
0,197336.103280
1,308229.865650
2,712497.390968
3,621433.773259
4,182054.558760
5,598472.220916
6,424749.619320
7,293260.871203
8,617019.686749
9,159761.223317
